In [2]:
import yfinance as yf

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime as dt

In [4]:
ibr50 = pd.read_csv('IBXLDia_19-08-24.csv',sep=';',encoding='latin1')
stock_list = [stock + '.SA' for stock in ibr50.iloc[:,0].to_list()]
data = yf.download(stock_list, start='2015-01-01',interval='1d',rounding=True)
out = data.stack().reset_index(level=1, names=["", "Ticker"]).sort_values(by=["Ticker",'Date']).reset_index()

[*********************100%***********************]  48 of 48 completed


In [5]:
out['AdjRatio'] = round(out['Adj Close']/out['Close'],4)
out['AdjHigh'] = round(out['AdjRatio']*out['High'],2)
out['AdjLow'] = round(out['AdjRatio']*out['Low'],2)
out['AdjOpen'] = round(out['AdjRatio']*out['Open'],2)
out['Low2High'] = round(out['AdjHigh']/out['AdjLow']-1,4)*100
out['Open2Close'] = round(out['Adj Close']/out['AdjOpen']-1,4)*100
out['CloseChange'] = out['Adj Close'].pct_change()
out['VolumeChange'] = out['Volume'].pct_change()

out.drop(['Close','High','Low','Open'],axis=1,inplace=True)
out['CloseChange'] = out.groupby('Ticker')['Adj Close'].pct_change()
out['VolumeChange'] = out.groupby('Ticker')['Volume'].pct_change()

# Adding More Complex Columns

In [46]:
def arithmetic_moving_average(change_col,window=15):
    return change_col.rolling(window).mean()

def geometric_moving_average(change_col, window=15):
    def geometric_mean(window_data):
        window_data = window_data / 100
        product = np.prod(1 + window_data)
        return (product ** (1 / len(window_data)) - 1) * 100
    return change_col.rolling(window=window).apply(geometric_mean, raw=True)

def stdv_moving(change_col,window=15):
    return change_col.rolling(window).std()

def semivariance_moving(change_col, window=15):
    def semivariance(window_data):
        below_values = window_data[window_data < 0]
        if len(below_values) > 1:
            return (((below_values - 0) ** 2).sum() / (len(below_values)-1))**(1/2)
        else:
            return 0
    return change_col.rolling(window=window).apply(semivariance, raw=True)

def min_return_moving(change_col,window=15):
    def min_return(window_data):
        return window_data.min()
    return change_col.rolling(window).apply(min_return, raw=True)

def drawdown(change_col):
    cumulative_returns = (1 + change_col).cumprod()
    running_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - running_max) / running_max
    return drawdown

import pandas as pd

def drawdown_duration(change_col):
    cumulative_returns = (1 + change_col).cumprod()
    running_max = cumulative_returns.cummax()
    drawdown_duration = pd.Series(0, index=change_col.index)
    count = 0
    for i in range(1, len(cumulative_returns)):
        if cumulative_returns.iloc[i] < running_max.iloc[i]:
            count += 1
        else:
            count = 0
        drawdown_duration.iloc[i] = count
    return drawdown_duration


def cumulative_returns(change_col):
    return np.cumprod(1+change_col)

In [47]:
out['CloseChangeCumulative'] = out.groupby('Ticker',group_keys=False)['CloseChange'].apply(cumulative_returns)
out['CloseChangeMA'] = out.groupby('Ticker',group_keys=False)['Adj Close'].apply(arithmetic_moving_average)
out['CloseChangeGMA'] = out.groupby('Ticker',group_keys=False)['Adj Close'].apply(geometric_moving_average)
out['CloseChangeSTD'] = out.groupby('Ticker',group_keys=False)['CloseChange'].apply(stdv_moving)
out['CloseChangeSMV'] = out.groupby('Ticker',group_keys=False)['CloseChange'].apply(semivariance_moving)
out['DrawdownMoving'] = out.groupby('Ticker',group_keys=False)['CloseChange'].apply(drawdown)
out['DrawdownDuration'] = out.groupby('Ticker',group_keys=False)['CloseChange'].apply(drawdown_duration)

In [53]:
plot_df

,Ticker,Adj Close,Volume,AdjRatio,AdjHigh,AdjLow,AdjOpen,Low2High,Open2Close,CloseChange,VolumeChange,CloseChangeCumulative,CloseChangeMA,CloseChangeGMA,CloseChangeSTD,CloseChangeSMV,DrawdownMoving,DrawdownDuration
Date,,,,,,,,,,,,,,,,,,
2015-01-02,VALE3.SA,11.64,5658400.0,0.5470,12.01,11.52,11.87,4.25,-1.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-01-05,VALE3.SA,11.46,8603000.0,0.5468,11.55,11.21,11.35,3.03,0.97,-0.015464,0.520394,0.984536,NaN,NaN,NaN,NaN,0.000000,0
2015-01-06,VALE3.SA,11.92,9879900.0,0.5468,12.10,11.52,11.52,5.03,3.47,0.040140,0.148425,1.024055,NaN,NaN,NaN,NaN,0.000000,0
2015-01-07,VALE3.SA,12.36,6130900.0,0.5469,12.47,12.02,12.21,3.74,1.23,0.036913,-0.379457,1.061856,NaN,NaN,NaN,NaN,0.000000,0
2015-01-08,VALE3.SA,12.49,4667300.0,0.5468,12.51,12.09,12.45,3.47,0.32,0.010518,-0.238725,1.073024,NaN,NaN,NaN,NaN,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,VALE3.SA,58.05,17168700.0,1.0000,58.55,58.05,58.50,0.86,-0.77,0.011324,-0.285194,4.987113,57.088667,57.087042,0.009508,0.008755,-0.318022,394
2024-08-27,VALE3.SA,59.80,35232600.0,1.0000,60.14,59.09,59.70,1.78,0.17,0.030146,1.052141,5.137457,57.254667,57.251594,0.012086,0.008755,-0.297462,395
2024-08-28,VALE3.SA,59.37,21280300.0,1.0000,59.50,58.68,59.37,1.40,0.00,-0.007191,-0.396005,5.100515,57.399333,57.395385,0.012310,0.009201,-0.302514,396


In [64]:
plot_df = out[(out.Ticker == 'VALE3.SA')].copy()
plot_df.set_index('Date',inplace=True)

fig = go.Figure()
fig = make_subplots(rows=5, cols=1, shared_xaxes=True,row_heights=[0.2, 0.2 ,0.2, 0.2,0.2],vertical_spacing=0.02,
                subplot_titles=('Cumulative Change', 'MA x GMA', 'STD x SMV','Drawdown','Drawdown Duration'))

# Add first subplot (CloseChange)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['CloseChangeCumulative'],
                         mode='lines',
                         name='Cumulative',
                         line=dict(color='blue'),
                         hoverinfo='x+y'),
              row=1, col=1)

# Add second subplot (CloseChangeSTD)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Adj Close'],
                         mode='lines',
                         name='MA',
                         line=dict(color='#1dc4c2'),
                         hoverinfo='x+y'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['CloseChangeMA'],
                         mode='lines',
                         name='MA',
                         line=dict(color='#34eb61'),
                         hoverinfo='x+y'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['CloseChangeGMA'],
                         mode='lines',
                         name='GMA',
                         line=dict(color='#096921'),
                         hoverinfo='x+y'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['CloseChangeSTD'],
                         mode='lines',
                         name='STD',
                         line=dict(color='#bfa12a'),
                         hoverinfo='x+y'),
              row=3, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['CloseChangeSMV'],
                         mode='lines',
                         name='SMV',
                         line=dict(color='#5e4e0b'),
                         hoverinfo='x+y'),
              row=3, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['DrawdownMoving'],
                         mode='lines',
                         name='CloseChangeSMV',
                         line=dict(color='#de1d1d'),
                         hoverinfo='x+y'),
              row=4, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['DrawdownDuration'],
                         mode='lines',
                         name='CloseChangeSMV',
                         line=dict(color='#780b0b'),
                         hoverinfo='x+y'),
              row=5, col=1)

# Update layout for subplots
fig.update_layout(
    title='Stock Data Analysis',
    height=1200,
    width=1600,
    showlegend=True
)

# Adjust layout for subplots
fig.update_layout(
    grid=dict(rows=3, columns=1),
)

# Show the figure
fig.show()